In [1]:
import plotly.express as px
from sqlalchemy import create_engine, engine
import pandas as pd
import numpy as np
import boto3
import os
from dotenv import dotenv_values, load_dotenv


In [2]:
data = ["2023-05-22 06:45:34.22222", "2019-05-22 15:45:34.22222", "2015-05-22 20:45:34.22222", "2010-05-22 14:45:34.22222"]
riders_day_df = pd.DataFrame(data, columns=['start_time'])
total = riders_day_df["start_time"].count()
print_line = f"Total number of riders in the last 24 hours: {total}"


In [3]:
data = [["2023-05-22 06:45:34.22222", "male"], ["2019-05-22 15:45:34.22222", "male"], ["2015-05-22 20:45:34.22222", "female"], ["2010-05-22 14:45:34.22222", "male"]]
riders_day_gender_df = pd.DataFrame(data, columns=['start_time', 'gender'])
gender_count = riders_day_gender_df.groupby('gender')['start_time'].count().reset_index(name='count')
fig = px.bar(gender_count, x = 'gender', y = 'count', title="Gender split between riders in the last 24 hours")
# fig.update_layout(
#     xaxis_title='Gender',
#     yaxis_title='Count'
# )

In [4]:
data = [["2023-05-16 06:45:34.22222", "2000-04-22"], ["2023-05-16 15:45:34.22222", "1992-08-10"], ["2023-05-16 20:45:34.22222", "2004-06-14"], ["2023-05-16 14:45:34.22222", "1999-12-25"]]
riders_day_gender_df = pd.DataFrame(data, columns=['start_time', 'dob'])
riders_day_gender_df['start_time'] = pd.to_datetime(riders_day_gender_df['start_time'])
riders_day_gender_df['dob'] = pd.to_datetime(riders_day_gender_df['dob'])
riders_day_gender_df['age'] = ((riders_day_gender_df['start_time'] - riders_day_gender_df['dob']).dt.days.astype(float)) * 0.00273973 #converts to normal number
riders_day_gender_df['age'] = riders_day_gender_df['age'].apply(np.floor)
print(riders_day_gender_df)

                  start_time        dob   age
0 2023-05-16 06:45:34.222220 2000-04-22  23.0
1 2023-05-16 15:45:34.222220 1992-08-10  30.0
2 2023-05-16 20:45:34.222220 2004-06-14  18.0
3 2023-05-16 14:45:34.222220 1999-12-25  23.0


In [5]:
bins = [10, 20, 30, 40, 50, 60, 70, 105]
labels = ['10-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71+']
age_groups = pd.cut(riders_day_gender_df['age'], bins=bins, labels=labels, right=True)
age_group_count = riders_day_gender_df.groupby([age_groups])['start_time'].count().reset_index(name='count')
age_fig = px.bar(age_group_count, x = 'age', y = 'count', title="Age group split between riders in the last 24 hours")
# age_fig.update_layout(
#     xaxis_title='Age',
#     yaxis_title='Count'
# )

In [6]:
data = [["2023-05-16 06:45:34.22222", "84", "11.22896664"], ["2023-05-16 15:45:34.22222", "85", "12.54785501"], ["2023-05-16 20:45:34.22222", "63", "1.648772319"], ["2023-05-16 14:45:34.22222", "72", "8.962429663"]]
riders_day_reading_df = pd.DataFrame(data, columns=['start_time', 'avg_heart_rate', 'avg_power'])
heart_fig = px.scatter(riders_day_reading_df, x="start_time", y="avg_heart_rate", color="avg_heart_rate", title="Average heart rate in the past day")
heart_fig.update_layout(
xaxis_title ="Time",
yaxis_title ="Heart rate"
)
power_fig = px.scatter(riders_day_reading_df, x="start_time", y="avg_power", color="avg_power", title="Average power in the past day")
power_fig.update_layout(
xaxis_title ="Time",
yaxis_title ="Power"
)
with open('report_graph.html', 'w') as f:
    f.write(f'<html>\n<head>\n<title>Daily Report</title>\n<link rel="stylesheet" href="./style.css">\n</head>\n<body>\n<div class="heading">\n<h1>Daily report</h1>\n<p1>{print_line}</p1>\n</div>\n<div class="container" >')
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(age_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(heart_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(power_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write("\n</body>\n</html>")

print(print_line)

Total number of riders in the last 24 hours: 4


In [34]:
fig.write_image("gender_split.png")
age_fig.write_image("age.png")
heart_fig.write_image("heart.png")
power_fig.write_image("power.png")
html_content = f'<html>\n<head>\n<title>Daily Report</title>\n<link rel="stylesheet" href="./style.css">\n</head>\n<body>\n<div class="heading">\n<h1>Daily report</h1>\n<p1>{print_line}</p1>\n</div>\n<div class="container" ><img src="./gender_split.jpeg"></div>'

In [20]:
with open('report_graph_2.html', 'w') as f:
    f.write(html_content)
    f.write("\n</body>\n</html>")

In [24]:
load_dotenv()
config = os.environ
session = boto3.Session(aws_access_key_id=config["ACCESS_KEY"], aws_secret_access_key=config["SECRET_KEY"])
ses = session.client("ses")
email_recipients = {"ToAddresses": ["trainee.dani.rahulan@sigmalabs.co.uk"], "CcAddresses": [], "BccAddresses": []}
print("before")
# pdfkit.from_file("./report_graph.html", "./report_graph.pdf")
# print("here")
with open("./gender_split.png", 'rb') as file:
    png_data = file.read()
# png_data_encoded = png_data.encode('utf-8')
message = {"Subject": {"Data": "File attached"}, "Body": {"Html": {"Data": html_content,}},'Attachments' :[{'Filename': 'gender_split.png', 'Content':png_data, 'ContentType': 'image/png', },]}
# message['Attachments'] =  [{'Filename': 'gender_split.png', 'Content':png_data, 'ContentType': 'image/png', },]
ses.send_email(Source="trainee.dani.rahulan@sigmalabs.co.uk", Destination=email_recipients, Message=message)


before


ParamValidationError: Parameter validation failed:
Unknown parameter in Message: "Attachments", must be one of: Subject, Body

In [36]:
import boto3
from botocore.exceptions import ClientError
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
load_dotenv()
config = os.environ
region = "eu-west-2"
sws_user = config["ACCESS_KEY"]
sws_key = config["SECRET_KEY"]
subject = 'Sending email with Attachment '
body = html_content

client = boto3.client(service_name = 'ses', region_name = region, aws_access_key_id = sws_user, aws_secret_access_key = sws_key)

   
message = MIMEMultipart()
message['Subject'] = subject
message['From'] = "trainee.dani.rahulan@sigmalabs.co.uk"
message['To'] = "trainee.dani.rahulan@sigmalabs.co.uk"

# message body
part = MIMEText(body, 'html')
message.attach(part)

attachmentUrl = "./gender_split.png"
with open("./gender_split.png", 'rb') as file:
    part = MIMEApplication(file.read())
    part.add_header("Content-Disposition",
                        "attachment",
                        filename=attachmentUrl)
message.attach(part)
with open("./age.png", 'rb') as file:
    part = MIMEApplication(file.read())
    part.add_header("Content-Disposition",
                        "attachment",
                        filename=attachmentUrl)
message.attach(part)
with open("./heart.png", 'rb') as file:
    part = MIMEApplication(file.read())
    part.add_header("Content-Disposition",
                        "attachment",
                        filename=attachmentUrl)
message.attach(part)
with open("./power.png", 'rb') as file:
    part = MIMEApplication(file.read())
    part.add_header("Content-Disposition",
                        "attachment",
                        filename=attachmentUrl)
message.attach(part)

   
destination = { 'ToAddresses' : [message['To']], 'CcAddresses' : [], 'BccAddresses' : []}

try:
    result = client.send_raw_email(Source = "trainee.dani.rahulan@sigmalabs.co.uk", Destinations = ["trainee.dani.rahulan@sigmalabs.co.uk", "trainee.mohammed.simjee@sigmalabs.co.uk", "trainee.ben.corrigan@sigmalabs.co.uk"], RawMessage = {'Data': message.as_string(),})
    print( {'message': 'error','status' : 'fail'} if 'ErrorResponse' in result else {'message': 'mail sent successfully', 'status' : 'success'})
except ClientError as e:
    print ({'message': e.response['Error']['Message'],'status' : 'fail'})

{'message': 'mail sent successfully', 'status': 'success'}
